In [1]:
import requests,re
import numpy as np
from bs4 import BeautifulSoup
import pdfkit

In [2]:
base_url = 'http://paulgraham.com'

In [3]:
test = requests.get(base_url+'/articles.html')
content = BeautifulSoup(test.content,'html.parser')

In [4]:
tables = content.findAll('table')

In [5]:
req_table_idx = 0;

In [6]:
for idx,table_content in enumerate(tables):
    if str(table_content).find('Essays') != -1:
        req_table_idx  = idx+1

In [7]:
req_table = tables[req_table_idx]

In [8]:
essay_owner = '<a href="http://paulgraham.com/index.html"><img src="https://sep.yimg.com/ca/I/paulgraham_2271_3232" width="410" vspace="0" hspace="0" height="45" border="0"></a><br/><br/>'
essay_title = '<img src="https://sep.yimg.com/ca/I/paulgraham_2272_1072" alt="Essays" width="57" vspace="0" hspace="0" height="18" border="0"><br/><br/>'

In [9]:
menu = essay_owner+essay_title+req_table.decode_contents()

In [10]:
rows = req_table.findAll('tr')

In [11]:
essay_url_list = []
essay_title_list = []
for row in rows:
    try:
        if len(row.select('a')) > 0:
            essay_info = row.select('a')[0]
            essay_title_list.append(essay_info.decode_contents())
            essay_url_list.append(essay_info['href'])
    except:
            print('error')

In [12]:
def essay_links_fix(essay_content,idx):
    et = essay_content
    if(idx>=0):
        ct = re.sub("(=\")(\w+).html",r'\g<1>'+base_url+r'/\g<2>.html',essay_content)
        et = re.sub(r'href="#(\w+)"',r'href="#\g<1>{}z"'.format(str(idx)),ct)
        et = re.sub(r'name="(\w+)"',r'name="\g<1>{}z"'.format(str(idx)),et)
        et = re.sub(r'#dddddd',r'#a2a2a2',et)
        et = re.sub(r'((<br.>){3,})<b>(.*?)</b>',r'<br/><br/><b>\g<3></b>',et)
    return et

In [13]:
start_template = '''<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
</head>
<body>'''
end_template = '''</body></html>'''

In [14]:
essay_list = [start_template+menu+end_template]

In [15]:
def replace_url(essay_url,ref_id):
#     print('ref:',ref_id)
    modified_url = essay_url[:]
    if(essay_url.find('?')!=-1) : 
        key = re.findall(r'\/(\w+.\w+)\?',essay_url)[0]
        if key:
#             print('key:',key)
            modified_url = re.sub(\
    r'<a href=("https://sep.yimg.com/ty/cdn/paulgraham/{}.*?")'.format(key),
    r'<a href="{}"'.format('#'+ref_id),essay_list[0])
    else:
        modified_url = re.sub(r'<a href=("{}")>'.format(essay_url),\
                           r'<a href="{}">'.format('#'+ref_id),essay_list[0])
    return modified_url

In [16]:
for idx,essay_url in enumerate(essay_url_list):
    try:
        print('adding essay no:'+str(idx+1)+' '+essay_title_list[idx])
#         req_url = base_url+'/'+essay_url
        if(len(re.findall(r'(http)',essay_url))==0):req_url = base_url+'/'+essay_url
        else:req_url=essay_url
#         print('essay url:',essay_url)
        ref_id = str(idx)+'_essay'      
        essay_list[0] = replace_url(essay_url,ref_id)
        essay_raw = requests.get(req_url)
        essay_parsed = BeautifulSoup(essay_raw.content,'html.parser')
        table_list = essay_parsed.findAll('table')
        
        if(len(table_list)>0): essay_content = table_list[1]
        else:essay_content = essay_parsed
        
        essay_content = essay_content.decode_contents()
        essay_content = essay_links_fix(essay_content,idx)
        if(essay_content.find('img')==-1):
#             print('idx:',idx)
            essay_content = '<img src="https://sep.yimg.com/ca/I/paulgraham_2272_1423"/>' + essay_content
        essay_content = re.sub(r'(<img .*?/>)',
            r'<a href="{}" id={} >'.format(req_url,ref_id)+'\g<1>'+'</a>',
            essay_content)
        formatted_content = start_template+\
            '<p class="break-before"><div id="title">'\
            + essay_content  + '</div>'+end_template
        essay_list.append(formatted_content)
    except:
        print('error in adding:',essay_title_list[idx])

adding essay no:1 Earnestness
adding essay no:2 Billionaires Build
adding essay no:3 How to Think for Yourself
adding essay no:4 Early Work
adding essay no:5 Modeling a Wealth Tax
adding essay no:6 The Four Quadrants of Conformism
adding essay no:7 Orthodox Privilege
adding essay no:8 Coronavirus and Credibility
adding essay no:9 How to Write Usefully
adding essay no:10 Being a Noob
adding essay no:11 Haters
adding essay no:12 The Two Kinds of Moderate
adding essay no:13 Fashionable Problems
adding essay no:14 Having Kids
adding essay no:15 The Lesson to Unlearn
adding essay no:16 Novelty and Heresy
adding essay no:17 The Bus Ticket Theory of Genius
adding essay no:18 General and Surprising
adding essay no:19 Charisma / Power
adding essay no:20 The Risk of Discovery
adding essay no:21 How to Make Pittsburgh a Startup Hub
adding essay no:22 Life is Short
adding essay no:23 Economic Inequality
adding essay no:24 The Refragmentation
adding essay no:25 Jessica Livingston
adding essay no:26

adding essay no:194 Chapter 2 of Ansi Common Lisp
adding essay no:195 Programming Bottom-Up
adding essay no:196 This Year We Can End the Death Penalty in California


In [17]:
options = {
   'footer-center': '-[page]-',
    'footer-font-size':'8'
   }

In [18]:
# pdfkit.from_string(' '.join([et,et2]),'earnet.pdf',options=options,css='test_css.css')
pdfkit.from_string(' '.join(essay_list),'pg_essay.pdf',options=options,css='test_css.css')

OSError: wkhtmltopdf reported an error:
Loading pages (1/6)
[>                                                           ] 0%[======>                                                     ] 10%[======================>                                     ] 37%[======================>                                     ] 37%[======================>                                     ] 38%[======================>                                     ] 38%[=======================>                                    ] 39%[=======================>                                    ] 39%Warning: Failed to load https://www.ycombinator.com/images/y18.gif (ignore)
[=========================>                                  ] 42%[=========================>                                  ] 42%[=========================>                                  ] 43%[============================>                               ] 47%[============================>                               